Jako zbiór danych wybrałem dane o COVID-19. Zbiór zaimportowałem do zasobów Azure ML w rozszerzeniu CSV. Azure automatyncznie dobiera typy danych, należy tylko wybrać pierwsze wiersze jako nazwy kolumn.
Eksperyment miał na celu przewidzenie sumy zachorowań w Polsce miesiąc od ostatnich wprowadzonych danych. Do tego celu za pomocą pipelines oddzieliłem z danych tylko wiersze dotyczące Polski.
Podczas pierwszej próby użyłem Zautomatyzowanego uczenia maszynowego, uczenie powiodło się i zwrócone zostały dane statystyczne.
Drugą próbą było wykorzystanie notebooka.


Pierwszym krokiem było zaimporotwanie bibliotek dla Azure ML Workspace i przygotowanie środowiska

In [25]:
import azureml.core

print("SDK version:", azureml.core.VERSION)

SDK version: 1.20.0


In [26]:
from azureml.core import Workspace, Dataset
ws = Workspace.from_config()

Następnie załadowane zostały dane, wybrałem tylko 2 potrzebne kolumny, ponieważ przy większej ilości występował błąd podczas uczenia.

In [52]:
columns = ['date','total_cases']
aml_dataset = ws.datasets['Poland'].keep_columns(columns, validate=False)
full_df = aml_dataset.to_pandas_dataframe()
full_df.head(5)

,date,total_cases
0,2020-03-04,1
1,2020-03-05,1
2,2020-03-06,5
3,2020-03-07,5
4,2020-03-08,11


In [28]:
full_df.describe()

,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,total_cases_per_million,new_cases_per_million,new_cases_smoothed_per_million,total_deaths_per_million,...,aged_65_older,aged_70_older,gdp_per_capita,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,hospital_beds_per_thousand,life_expectancy,human_development_index
count,3.280000e+02,328.000000,323.000000,320.000000,320.000000,323.000000,328.000000,328.00000,323.000000,320.000000,...,3.280000e+02,3.280000e+02,3.280000e+02,3.280000e+02,3.280000e+02,3.280000e+02,3.280000e+02,3.280000e+02,3.280000e+02,3.280000e+02
mean,3.194374e+05,4506.460366,4529.488737,6952.284375,110.628125,107.618307,8440.319104,119.07175,119.680189,183.696388,...,1.676300e+01,1.020200e+01,2.721645e+04,2.273310e+02,5.910000e+00,2.330000e+01,3.310000e+01,6.620000e+00,7.873000e+01,8.650000e-01
std,4.792474e+05,6949.164790,6758.186515,10140.638816,174.673549,152.403750,12662.889413,183.61395,178.567843,267.940525,...,6.404655e-14,8.717447e-14,1.238803e-10,3.985119e-13,3.291281e-14,1.245350e-13,1.992559e-13,5.515120e-14,3.700467e-13,3.891718e-15
min,1.000000e+00,0.000000,2.286000,1.000000,0.000000,0.000000,0.026000,0.00000,0.060000,0.026000,...,1.676300e+01,1.020200e+01,2.721644e+04,2.273310e+02,5.910000e+00,2.330000e+01,3.310000e+01,6.620000e+00,7.873000e+01,8.650000e-01
25%,2.155475e+04,318.750000,340.214000,1063.250000,10.000000,10.714000,569.529250,8.42225,8.989000,28.093250,...,1.676300e+01,1.020200e+01,2.721644e+04,2.273310e+02,5.910000e+00,2.330000e+01,3.310000e+01,6.620000e+00,7.873000e+01,8.650000e-01
50%,5.570450e+04,595.000000,606.429000,1904.500000,18.000000,16.143000,1471.849000,15.72100,16.023000,50.321500,...,1.676300e+01,1.020200e+01,2.721644e+04,2.273310e+02,5.910000e+00,2.330000e+01,3.310000e+01,6.620000e+00,7.873000e+01,8.650000e-01
75%,4.463218e+05,7006.500000,8139.714500,7374.250000,130.500000,235.500000,11792.914000,185.12925,215.071000,194.845500,...,1.676300e+01,1.020200e+01,2.721644e+04,2.273310e+02,5.910000e+00,2.330000e+01,3.310000e+01,6.620000e+00,7.873000e+01,8.650000e-01
max,1.478119e+06,32733.000000,25611.000000,35401.000000,674.000000,505.286000,39055.524000,864.88600,676.705000,935.381000,...,1.676300e+01,1.020200e+01,2.721644e+04,2.273310e+02,5.910000e+00,2.330000e+01,3.310000e+01,6.620000e+00,7.873000e+01,8.650000e-01


By można było przeprowadzić uczenie konieczne jest połączenie z clusterem obliczeniowym.

In [29]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget

ComputeTarget.list(ws)

[{
   "id": "/subscriptions/d6ac2b6d-798c-4c8b-aa46-2927fa201cd5/resourceGroups/ml/providers/Microsoft.MachineLearningServices/workspaces/machinelearning/computes/testMLchmury",
   "name": "testMLchmury",
   "location": "northeurope",
   "tags": null,
   "properties": {
     "description": null,
     "computeType": "ComputeInstance",
     "computeLocation": "northeurope",
     "resourceId": null,
     "provisioningErrors": null,
     "provisioningState": "Succeeded",
     "properties": {
       "vmSize": "STANDARD_DS2_V2",
       "applications": [
         {
           "displayName": "Jupyter",
           "endpointUri": "https://testmlchmury.northeurope.instances.azureml.ms"
         },
         {
           "displayName": "Jupyter Lab",
           "endpointUri": "https://testmlchmury.northeurope.instances.azureml.ms/lab"
         },
         {
           "displayName": "RStudio",
           "endpointUri": "https://testmlchmury-8787.northeurope.instances.azureml.ms"
         }
       ]

In [30]:
amlcompute_cluster_name = "testMLchmury"

cts = ws.compute_targets

aml_remote_compute = cts[amlcompute_cluster_name]

In [31]:
aml_remote_compute.get_status().serialize()

{'errors': [],
 'creationTime': '2021-01-26T14:33:17.849682+00:00',
 'createdBy': {'userObjectId': '1d560a7f-09a2-41c2-9ddc-ec1db76ad069',
  'userTenantId': '3b50229c-cd78-4588-9bcf-97b7629e2f0f',
  'userName': 'Gawieńczuk Adam (STUD)'},
 'modifiedTime': '2021-01-26T14:36:04.183841+00:00',
 'state': 'Running',
 'vmSize': 'STANDARD_DS2_V2'}

Import bibliotek AutoML

In [32]:
from azureml.train import automl

In [33]:
from azureml.train.automl import AutoMLConfig

Konfiguracja uczenia, poprzez podanie kolumny z czasem, zakresem przewidywania, typem zadania, algorytmem uczącym, zbiorem danych i innymi potrzebnymi danymi.

In [71]:
time_series_settings = {
    'time_column_name': 'date',
    'forecast_horizon': 30
}

automl_config = AutoMLConfig(task = 'forecasting',
                             debug_log='debug1.log',
                             primary_metric='normalized_root_mean_squared_error',
                             experiment_timeout_minutes=20,
                             training_data=aml_dataset,
                             label_column_name='total_cases',
                             n_cross_validations=5,
                             path='./normalized_root_mean_squared_error',
                             verbosity=logging.INFO,
                             **time_series_settings)

Po skonfigurowaniu uczenia może zostać ono rozpoczęte.

In [54]:
from azureml.core import Experiment
from datetime import datetime

now = datetime.now()
time_string = now.strftime("%m-%d-%Y-%H")
experiment_name = "normalized_root_mean_squared_error-{0}".format(time_string)
print(experiment_name)

experiment = Experiment(workspace=ws, name=experiment_name)

import time
start_time = time.time()
            
run = experiment.submit(automl_config, show_output=True)


print('Manual run timing: --- %s seconds needed for running the whole Remote AutoML Experiment ---' % (time.time() - start_time))

normalized_root_mean_squared_error-01-26-2021-23
No run_configuration provided, running on local with default configuration
Running on local machine
Parent Run ID: AutoML_b24b44d6-2616-4522-ae7b-04c865b6125f

Current status: DatasetFeaturization. Beginning to featurize the dataset.
Current status: DatasetFeaturizationCompleted. Completed featurizing the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: DatasetFeaturization. Beginning to featurize the CV split.
Current status: DatasetFeaturizationCompleted. Completed featurizing the CV split.
Current status: DatasetFeaturization. Beginning to featurize the CV split.
Current status: DatasetFeaturizationCompleted. Completed featurizing the CV split.
Current status: DatasetFeaturization. Beginning to featurize the CV split.
Current status: DatasetFeaturizationCompleted. Completed featurizing the CV split.
Current status: DatasetFeaturization. Beginning to featurize the CV sp

Wybór najlepszego przebiegu do testu działania nauczonej sieci.

In [62]:
best_run, fitted_model = run.get_output()

Test się nie powiódł, nie udało się wykonać przewidywania na danych testowych i zwrócenia danych w postaci wykresu.